# Check source calibration

- author : Sylvie Dagoret-Campagne
- creation date : 2025-06-06
- last update  : 2025-06-07

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import astropy.units as u
from astropy.stats import sigma_clipped_stats

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 16})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 5})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})


In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
all_bands_colors = ["blue", "green", "red", "orange", "yellow", "purple"]

In [ ]:
def nJy_to_ab_mag(f_njy):
    """Convert scalar or array flux in nJy to AB magnitude."""
    f_njy = np.asarray(f_njy)
    mag = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = f_njy > 0
    mag[mask] = -2.5 * np.log10(f_njy[mask]) + 31.4
    return mag


def nJy_err_to_ab_err(f_njy, f_err):
    """Propagate flux error to magnitude error."""
    f_njy = np.asarray(f_njy)
    f_err = np.asarray(f_err)
    mag_err = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = (f_njy > 0) & (f_err > 0)
    mag_err[mask] = (2.5 / np.log(10)) * (f_err[mask] / f_njy[mask])
    return mag_err


In [ ]:
file = "all_src_t5063_p14_magcutg25_psf_ap.csv"
ap_radius = [ '03','06','09','12','17','25','35','50','70' ]

## Input file read and process

In [ ]:
df = pd.read_csv(file)

In [ ]:
print(list(df.columns))

### Pre-compute usefull quantities like mags, flux ratio and differences en mag differences

In [ ]:
psfflux_name = "psfFlux"
psffluxerr_name = "psfFluxErr"
psfmag_name = "psfMag"
psfmagerr_name = "psfMagErr"            
df[psfmag_name] = df[psfflux_name].apply(nJy_to_ab_mag)
df[psfmagerr_name] = nJy_err_to_ab_err(df[psfflux_name], df[psffluxerr_name])

In [ ]:
calibflux_name = "calibFlux"
calibfluxerr_name = "calibFluxErr"
calibmag_name = "calibMag"
calibmagerr_name = "calibMagErr"            
df[calibmag_name] = df[calibflux_name].apply(nJy_to_ab_mag)
df[calibmagerr_name] = nJy_err_to_ab_err(df[calibflux_name], df[calibfluxerr_name])

In [ ]:
df["ratio_calibFlux"] = df["calibFlux"]/df["psfFlux"] 
df["diff_calibFlux"] = df["calibFlux"]-df["psfFlux"] 
df["diff_calibMag"] = (df[calibmag_name]  -  df[psfmag_name])*1000.  
df["ratiodiff_calibFlux"] = df["diff_calibFlux"]/df["psfFlux"]

In [ ]:
for radius in ap_radius:
    fluxName = f"ap{radius}Flux"
    magName = f"ap{radius}Mag"

    # compute the magnitude
    df[magName] = df[fluxName].apply(nJy_to_ab_mag)

    # define the columns names
    diff_fluxname = "diff_" + fluxName
    ratio_name = "ratio_" + fluxName
    diffratio_name = "ratiodiff_" + fluxName

     # calculate the ratio and difference for Ap Fluxes 
    df[ratio_name] =  df[fluxName]/df["psfFlux"] 
    df[diff_fluxname] = df[fluxName] - df["calibFlux"]
    df[diffratio_name] = df[diff_fluxname]/df["psfFlux"] 

    # On Ap magnitudes:
    # to find the closest aperture flux to the psf or calib
    diff_magname_psf = "diff_" + magName + "_psf"
    diff_magname_calib= "diff_" + magName + "_calib"

    # calculate the difference for magnitudes
    df[diff_magname_psf] = (df[magName] - df[psfmag_name])*1000.
    df[diff_magname_calib] = (df[magName] - df[calibmag_name])*1000.


In [ ]:
print(list(df.columns))

### Separate in different bands and cut on Mag
- mag cut to avoid large tail at low flux preventing viewing histograms

In [ ]:
MAGCUT = 20
df = df[df[psfmag_name] < MAGCUT]

In [ ]:
all_df = []
for band in all_bands:
    the_df = df[df["band"] == band]
    all_df.append(the_df)

## Define Plots

In [ ]:
def plot_scatter_perband(x,y,nrows=2,ncols=3,scale = "log",bisectriss = True, aspect="equal"):
    fig,axes = plt.subplots(nrows,ncols,figsize=(16,10),layout="constrained")
    axs = axes.flatten()

    for idx,band in enumerate(all_bands):
        
        ax = axs[idx]
        the_color = all_bands_colors[idx]
        the_df = all_df[idx]

        the_x = the_df[x].values
        the_y = the_df[y].values

        the_x = the_x[~np.isnan(the_x)]
        the_y = the_y[~np.isnan(the_y)]

        min_val = min(np.min(the_x), np.min(the_y))
        max_val = max(np.max(the_x), np.max(the_y))

        if bisectriss :
            ax.plot([min_val, max_val], [min_val, max_val], 'k--',lw=3)
        else:
            ax.axhline(1,color="k")


        the_df.plot.scatter(x=x,y=y,ax=ax,marker="o",color=the_color,label=band,alpha=0.5)

        
        ax.legend()
        ax.grid()
        #ax.set_xlim(min_val,max_val)
        #ax.set_ylim(min_val,max_val)
        ax.set_yscale(scale)
        ax.set_xscale(scale)
        ax.set_aspect(aspect)
        
    suptitle = f"{y} vs {x}"
    plt.suptitle(suptitle,fontsize=16,fontweight="bold")
    plt.show()


In [ ]:
def plot_histo_perband(x,bins=50, nrows=2,ncols=3,scale = "log", aspect="auto",unit = "mmag"):
    fig,axes = plt.subplots(nrows,ncols,figsize=(16,10),layout="constrained")
    axs = axes.flatten()

    stats_per_band = {}  # ou [] si tu préfères construire une liste de dicts
    for idx,band in enumerate(all_bands):
        
        ax = axs[idx]
        the_color = all_bands_colors[idx]
        the_df = all_df[idx]

        the_x = the_df[x].values
        the_x = the_x[~np.isnan(the_x)]

        mean, median, std = sigma_clipped_stats(the_x, sigma=3.0, maxiters=5)
        
        
        # Ajouter un encadré avec les statistiques
        stats_text = f"mean = {mean:.2f} {unit} \nmedian = {median:.2f} {unit} \nσ = {std:.2f} {unit}"
        ax.text(
            0.6, 0.85,
            stats_text,
            transform=ax.transAxes,
            fontsize=12,
            verticalalignment='bottom',
            horizontalalignment='left',
            bbox=dict(facecolor='white', alpha=0.8, edgecolor='gray')
            )
        stats_per_band[band] = {
            "mean": mean,
            "median": median,
            "sigma": std
        }
            
        ax.hist(the_x,bins=bins,range=(median- 5*std, median + 5*std),color = the_color, label = band)
    
        
        ax.legend(loc="upper left")
        ax.grid()

        if scale == "log":
            ax.set_yscale(scale)
        #ax.set_xscale(scale)
        ax.set_aspect(aspect)

    stats_df = pd.DataFrame.from_dict(stats_per_band, orient='index')
    stats_df.index.name = "band"
        
    suptitle = f"{x}"
    plt.suptitle(suptitle,fontsize=16,fontweight="bold")
    plt.show()
    return  stats_df


## Run the plots

### 2D scatter plots 

In [ ]:
plot_scatter_perband(x= "psfFlux",y= "calibFlux")

In [ ]:
plot_scatter_perband(x= "psfFlux",y= "ratio_calibFlux",bisectriss = False, aspect="auto")

## Aperture Fluxes

In [ ]:
for radius in ap_radius:
    fluxName = f"ap{radius}Flux"
    ratio_name = "ratio_" + fluxName

    plot_scatter_perband(x= "psfFlux",y = fluxName)
    plot_scatter_perband(x= "psfFlux",y= ratio_name,bisectriss = False, aspect="auto")

### Histograms

In [ ]:
stats_df_apcalib = plot_histo_perband(x="diff_calibFlux",bins=50, nrows=2,ncols=3,scale = "linear", aspect="auto",unit="nJ")

In [ ]:
stats_df_calib_psf = plot_histo_perband(x="diff_calibMag",bins=50, nrows=2,ncols=3,scale = "log", aspect="auto",unit="mmag")

## Plot magnitude difference Ap-mag - ref Mag  and make some statistics

### Magnitude difference Ap-mag - psf Mag 

In [ ]:
all_stat_ap_psf = {}

for radius in ap_radius:
    magName = f"ap{radius}Mag"
    diff_magname_psf = "diff_" + magName + "_psf"
    #diff_magname_calib= "diff_" + magName + "_calib"
    
    diff_name = "diff_" + magName
    stats_df_ap = plot_histo_perband(x=diff_magname_psf,bins=50, nrows=2,ncols=3,scale = "log", aspect="auto",unit="mmag")
    all_stat_ap_psf[radius] = stats_df_ap

### Magnitude difference Ap-mag - calib Mag 

In [ ]:
all_stats_ap_calib = {}

for radius in ap_radius:
    magName = f"ap{radius}Mag"
    #diff_magname_calib = "psf_" + magName + "_psf"
    diff_magname_calib= "diff_" + magName + "_calib"
    
   
    stats_df_ap = plot_histo_perband(x=diff_magname_calib,bins=50, nrows=2,ncols=3,scale = "log", aspect="auto",unit="mmag")
    all_stats_ap_calib[radius] = stats_df_ap

In [ ]:
#all_stat_ap_psf

In [ ]:
#all_stat_ap_calib

In [ ]:
all_stat_ap_calib.keys()

In [ ]:
radii = sorted([float(k) for k in all_stat_ap_calib.keys()])

In [ ]:
all_stat_ap_calib['12']

In [ ]:
# Initialiser les courbes
fig, ax = plt.subplots(figsize=(10, 8))

for ib, band in enumerate(all_bands):
    medians = []
    sigmas = []
    valid_radii = []
    the_color = all_bands_colors[ib]
    
    for radius_str, df in all_stat_ap_psf.items():
        
        if band in df.index:
            medians.append(df.loc[band, 'median'])
            sigmas.append(df.loc[band, 'sigma'])
            valid_radii.append(float(radius_str))
    
    ax.errorbar(valid_radii, medians, yerr=sigmas, label=band, marker='o', color= the_color,capsize=3)

ax.set_xlabel('Radius R (pixels)')
ax.set_ylabel('apMag(R) - psfMag (mmag)')
ax.set_title('Magnitude difference (Aperture - Psf) vs radius')
ax.legend(title='Band')
ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Initialiser les courbes
fig, ax = plt.subplots(figsize=(10, 8))

for ib, band in enumerate(all_bands):
    medians = []
    sigmas = []
    valid_radii = []
    the_color = all_bands_colors[ib]
    
    for radius_str, df in all_stat_ap_calib.items():
        
        if band in df.index:
            medians.append(df.loc[band, 'median'])
            sigmas.append(df.loc[band, 'sigma'])
            valid_radii.append(float(radius_str))
    
    ax.errorbar(valid_radii, medians, yerr=sigmas, label=band, marker='o', color= the_color,capsize=3)

ax.set_xlabel('Radius R (pixels)')
ax.set_ylabel('apMag(R) - calibMag (mmag)')
ax.set_title('Magnitude difference (Aperture - Calib) vs radius')
ax.legend(title='Band')
ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
medians